In [ ]:
import numpy as np
import tensorflow as tf
from model import Model
import os

In [ ]:
with open('data.txt') as f:
    first = f.readline()
    first = first.strip('\n')
    temp = first.split(' ')
    T = int(temp[0])
    o_dim = int(temp[1])
    s_dim = int(temp[2])
    o_matrix = np.zeros((T, o_dim), np.float32)
    for i in range(T):
        temp = f.readline().strip('\n').split(' ')
        for j in range(s_dim):
            o_matrix[i,j] = float(temp[j])
    s_matrix = np.zeros((T, s_dim), np.float32)
    for i in range(T):
        temp = f.readline().strip('\n').split(' ')
        for j in range(o_dim):
            s_matrix[i,j] = float(temp[j])

In [ ]:
h_dim = 10
train_data = np.zeros((int(T/2), 3*s_dim), np.float32)
for i in range(int(T/2)):
    train_data[i, :] = np.concatenate((s_matrix[i, :], s_matrix[i+1, :], o_matrix[i+1, :]), axis=0)
test_data = np.zeros((int(T/2) - 1, 3*s_dim), np.float32)
for i in range(int(T/2) - 1):
    test_data[i, :] = np.concatenate((s_matrix[int(T/2) + i, :], s_matrix[int(T/2) + 1 + i, :], o_matrix[int(T/2) + 1 + i, :]),0)

In [ ]:
class Dataset(object):
    def __init__(self, train, test):
        self.train = train
        self.test = test
    def random_batch(self, batch_size):
        index = np.random.choice(np.arange(len(self.train)),batch_size, False)
        return self.train[index,:]
dataset = Dataset(train_data, test_data)

In [ ]:
log_dir = './log/'
os.popen('rm '+log_dir+'*')
minibatch_size = 256
model = Model(s_dim, h_dim, minibatch_size, 1e-4, log_dir)
iteration = 300
for epoch in range(500):
    reconstruction_loss_train, classify_loss_train = 0., 0.
    global_step = tf.contrib.framework.get_or_create_global_step()
    # in each epoch 500 iterations
    for i in range(iteration):
        reconstruction_loss_value, classify_loss_value, summary = \
                model.update_params(dataset.random_batch(minibatch_size))
        classify_loss_train += classify_loss_value
        reconstruction_loss_train += reconstruction_loss_value
        model.train_writer.add_summary(summary, global_step.eval(model.sess))
    
    classify_loss_train = classify_loss_train / (iteration)
    reconstruction_loss_train = reconstruction_loss_train / (iteration)

    print("step: {},\trecons loss: {:.4f},\tclass loss: {:.4f}".format(global_step.eval(model.sess),
            reconstruction_loss_train, classify_loss_train))
